### DNSmasq

If you would like to have DNS resolve certain DNS names to localhost for your cluster. This can be done with *dnsmasq*, which allows us to override wildcard DNS entries and point them to localhost.  

Since we're on a Mac, we're going to use 'brew' to install it:

<div class="alert alert-block alert-info">
    <b>Note: </b> 
If you do not already have Homebrew for MacOS, install this first using the following link:
https://brew.sh/
</div>

In [ ]:
brew install dnsmasq

In [ ]:
brew list | grep -i dnsmasq

<br>

In your Jupyter notebook working directory, create a file called "password" and put your root password in this file. 

<div class="alert alert-block alert-info">
    <b>Note: </b> 
    This is to enable Jupyter notebook to execute sudo commands without exposing your root password or storing it in your machines CLI history while creating a variable. It can be deleted afterwards and there is step included in this notebook to do so. 
</div>

You can see your Jupyter notebook working directory by executing the following:

In [ ]:
pwd

Next we will add a rule to the configuration for dnsmasq that routes any URL ending in *.k8s.example.com* to 127.0.0.1.  The code below checks first to see if that line already exists and adds it if it does not:

In [ ]:
if grep -Fxq "address=/.k8s.example.com/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
then
    echo "DNSMasq rule for .k8s.example.com top-level domain already exists"
else
    cat password | sudo -S echo "address=/.k8s.example.com/127.0.0.1" >> /opt/homebrew/etc/dnsmasq.conf
    grep "address=/.k8s.example.com/127.0.0.1" /opt/homebrew/etc/dnsmasq.conf
fi

<br>

From here we will need to use *brew* to restart the dnsmasq service so that the new configuration can be loaded & used:

In [ ]:
cat password | sudo -S brew services restart dnsmasq

<br>

Use `dig` to test that URLs ending in `.k8s.example.com` resolve to 127.0.0.1 on the DNS server running at 127.0.0.1.  The `@127.0.0.1` option tells dig to use the DNS server running on localhost, the `dnsmasq` service we started earlier.  Note that the `ANSWER SECTION` provides the response for `fubar.k8s.example.com` as `127.0.0.1`.

In [ ]:
dig @127.0.0.1 fubar.k8s.example.com

<br>

Next, we'll want to override the Mac's DNS resolver for all URLs ending in `.k8s.example.com`, sending only those URLs to our dnsmasq service.  The mac uses "/etc/resolver" for this purpose.  This will allow us to use any browser, such as Chrome or Safari, to use `dnsmasq` to resolve URLs ending in `.k8s.example.com`.

If there is a file in `/etc/resolver` with a filename that matches the end of the URL, then the contents of that file will identify the DNS server to use for that URL.  In our case, we create a file named `/etc/resolver/k8s.example.com` that configures handling of all URLs ending in `.k8s.example.com`, a fictitious top-level domain that I created for test purposes.  This easily could be a subdomain of an actual domain owned by the cluster operator (for example, to listen to any URLs ending in `.k8s.example.com`, the full filename would be `/etc/resolver/k8s.example.com`

In [ ]:
cat password | sudo -S mkdir -p /etc/resolver

In [ ]:
if grep -Fxq "nameserver 127.0.0.1" /etc/resolver/k8s.example.com
then
    echo "MacOS DNS resolver for .k8s.example.com top-level domain already defers to localhost"
else
    cat password | sudo -S sh -c 'echo "nameserver 127.0.0.1" >> /etc/resolver/k8s.example.com'
    cat /etc/resolver/k8s.example.com
fi

### Remove Password File

In [ ]:
rm password

<br>

### Testing
We will test that the entire process works from end-to-end by deploying an NGINX pod to our local k8s cluster, exposing it using an Ingress resource, and then access the pod in our web browser:

In [ ]:
kubectl create deployment nginx --image=nginx

In [ ]:
kubectl create service clusterip nginx --tcp=80:80

In [ ]:
cat << EOF > ingress.yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: nginx
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
spec:
  rules:
  - host: nginx.k8s.example.com
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: nginx
            port:
              number: 80
EOF

In [ ]:
kubectl apply -f ingress.yaml

<br>

If you are following along with our other notebooks.

Recall that when we started our k3d cluster, we mapped the cluster's ingress controller service's ports 80 & 443 to localhost ports 8081 and 8443.  We also configured 'dnsmasq' to resolve all URLs ending in '.k8s.example.com' to 127.0.0.1.  This allows us to use our web browser to visit HTTP & HTTPS services exposed using ingress objects inside our local k3d cluster.

Open a browser window to [http://nginx.k8s.example.com:8081](http://nginx.k8s.example.com:8081) and confirm that you can see the NGINX starter page.  Likewise, you will be able to visit https://nginx.k8s.example.com:8443 (after accepting TLS certificate errors) and also see the same page.  Likewise, confirm that http://bad-url.k8s.example.com:8081 also resolves to the ingress controller, but gives a `404 page not found` error.
<br>

### Cleanup
After confirming that the cluster works as expected, let's remove our test instance of nginx:

In [ ]:
rm password

In [ ]:
kubectl delete ingress nginx

In [ ]:
kubectl delete service nginx

In [ ]:
kubectl delete deployment nginx